# Consolidação de Arquivos - Detalhes

In [1]:
# "latin1", "iso-8859-1", ou "cp1252",

import re
import os
from pathlib import Path

import duckdb
import openpyxl
import pandas as pd
from pandas import DataFrame

In [2]:
def _importar_arquivo(arquivo: str, aba: str=None, encoding: str ="latin1") -> DataFrame:
    try:
        if arquivo.endswith(".xlsx") or arquivo.endswith(".xls"):
            df = pd.read_excel(arquivo, engine="openpyxl", sheet_name=aba, dtype="str")
        elif arquivo.endswith(".csv") or arquivo.endswith(".txt"):
            df = pd.read_csv(arquivo, encoding=encoding, delimiter=";", dtype="str")
        elif arquivo.endswith(".json"):
            df = pd.read_json(arquivo)
        else:
            raise ValueError("Formato de arquivo não suportado")
        return df
    except Exception as erro:
        raise ValueError(f"Erro: {erro}")

In [3]:
def amostra_atributo(atributo: str, dataframe: DataFrame, num_amostras:int=2) -> str:
    try:
        query = f"""
        SELECT {atributo}
        FROM dataframe WHERE {atributo}
        IS NOT NULL
        LIMIT {num_amostras}
        """
        resultado_query = (
            [
                str(item[0])
                .replace(",", "")
                .strip()
                for item
                in duckdb
                .sql(query)
                .fetchall()
            ]
        )
        if len(resultado_query) == 0:
            resultado_query = ["SEM AMOSTRA"]
        return ";".join(resultado_query)
    except Exception as _:
        return "SEM AMOSTRA" 

In [4]:
def detalhes_arquivos(lista_arquivos: list) -> DataFrame:
    try:
        lista_temp = []
        for item in lista_arquivos:
            caminho = Path(os.path.relpath(item)).parent
            arquivo = f"{Path(os.path.abspath(item)).stem}{Path(os.path.abspath(item)).suffix}" 
            try:
                if item.endswith(".xlsx") or item.endswith(".xls"): 
                    abas_planilha = pd.ExcelFile(item).sheet_names

                    for aba in abas_planilha:
                        df = _importar_arquivo(item, aba)
                        colunas = [coluna for coluna in df.columns.tolist()]
                        quantidades = df.count().tolist()

                        for coluna, quantidade in zip(colunas, quantidades):
                            amostra = amostra_atributo(coluna, df)
                            lista_temp.append(
                                {   
                                    "caminho": caminho,
                                    "arquivo": arquivo,
                                    "coluna": coluna.strip(),
                                    "amostra": amostra,
                                    "quantidade": quantidade,
                                    "aba": aba,
                                    "observacoes": "",
                                    "status": "OK",
                                    "nova_coluna": (
                                        "***Esse atributo pode renomear "
                                        "as colunas na consolidação final.*** "
                                    )
                                    }
                            )
                else:
                    df = _importar_arquivo(item)
                    colunas = [coluna for coluna in df.columns.tolist()]
                    quantidades = df.count().tolist()
                    for coluna, quantidade in zip(colunas, quantidades):
                        amostra = amostra_atributo(coluna, df)
                        lista_temp.append(
                            {   
                                "caminho": caminho,
                                "arquivo": arquivo,
                                "coluna": coluna.strip(),
                                "amostra": amostra,
                                "quantidade": quantidade,
                                "aba": "",
                                "observacoes": "",
                                "status": "OK",
                                "nova_coluna": (
                                    "***Esse atributo pode renomear "
                                    "as colunas na consolidação final.*** "
                                )
                                }
                        )
            except Exception as erro:
                lista_temp.append(
                    {
                        "caminho": caminho,
                        "arquivo": arquivo,
                        "coluna": "",
                        "amostra": "",
                        "quantidade": "",
                        "aba": "",                        
                        "observacoes": f"{erro}",
                        "status": "ERRO",
                        "nova_coluna": ""
                        }
                )
        return pd.DataFrame(lista_temp)
    except Exception as erro:
        raise ValueError(f"Erro na geração dos detalhes. Erro: {erro}")

### Importando a Lista de arquivos a partir dos Metadados

In [5]:
metadados_arquivos = pd.read_csv("../resultados/metadados/consolidacao_metadados.csv", sep=";", dtype="str")

In [ ]:
metadados_arquivos

In [7]:
lista_arquivos = (
    [
        f"{caminho}/{arquivo}"
        for caminho, arquivo
        in zip(metadados_arquivos["caminho"], metadados_arquivos["arquivo"])
    ]
)

In [ ]:
lista_arquivos

In [9]:
df_detalhes_arquivos = detalhes_arquivos(lista_arquivos)

In [ ]:
df_detalhes_arquivos.sort_values(by=["arquivo","aba","coluna"])

In [11]:
(
    df_detalhes_arquivos.sort_values(by=["caminho","arquivo","aba","coluna"])
    .to_csv("../resultados/detalhes/consolidacao_detalhes.csv", index=False, sep=";")
)